# SVR model

## Import libraries and download the data

In [2]:
#Download libraries
import re
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import time
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from drop import drop_columns
from PCA_data import *
from imputer import *
from drop import *
from feature_eng import *
from sklearn import metrics
from PCA_data import *

In [3]:
#Download the data
df = pd.read_csv('../data/train_data.zip')
df.head()

,external_id,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,...,avg_wind_9_10,avg_wind_10_11,avg_wind_11_12,avg_wind_12_above,perfect_days,unacast_session_count,hpi,state_and_local_amount_per_capita,state_amount_per_capita,local_amount_per_capita
0,1804425,8,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,90.0,244.20,0.157475,0.009783,0.147692
1,1812706,2,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,27.0,258.95,0.157475,0.009783,0.147692
2,1812706,3,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,27.0,258.95,0.157475,0.009783,0.147692
3,1812706,11,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,24.0,258.95,0.157475,0.009783,0.147692
4,1812706,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,258.95,0.157475,0.009783,0.147692


In [4]:
# drop rows missing target variable
df = drop_missing_unacast(df)
# create X and y
X = df.drop('unacast_session_count', axis=1)
y = df.loc[:, 'unacast_session_count']
# split the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                    test_size=0.2,
                                                      random_state=2020)
# impute NaN values
result = impute_data(X_train, X_valid)
X_train = result[0]
X_valid = result[1] 
# perform feature eng
X_train = comb_cols(X_train)
X_valid = comb_cols(X_valid)
# perform dropping
X_train = drop_columns(X_train)
X_valid = drop_columns(X_valid)

# perform OHE (climate, density_class, income_class)
X_train_valid = clean_categorical(X_train, X_valid)
X_train = X_train_valid[0]
X_valid = X_train_valid[1]

In [5]:
X_train.head()

,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,monthly_count_ramp,...,avg_fertility_rate,HI,LI,MI,HD,LD,MD,A,C,D
28601,5,2018,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0,...,63.906875,0,1,0,0,1,0,0,1,0
27003,4,2019,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0,...,61.173125,1,0,0,0,0,1,0,0,1
36815,6,2018,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0,...,65.725000,0,1,0,0,1,0,0,0,1
27301,9,2019,0,0,0,0.000000e+00,0.000000,0.000000,0.000000,0,...,65.127500,1,0,0,1,0,0,0,0,1
1400,7,2019,176,16,160,1.540398e+06,438295.454545,156642.045455,56107.954545,0,...,58.470000,1,0,0,0,0,1,0,0,1


# SVR Model

## With preprocessed data

In [5]:
#Fit basic SVR with default values
t0 = time.time()
svr_init = SVR()
svr_init.fit(X_train, y_train.to_numpy().ravel())
t1 = time.time()

In [6]:
svr_init_train_time = t1 - t0
print('Training time : ', svr_init_train_time)

Training time :  2193.06933259964


In [9]:
#Find the train RMSE and MAE
svr_init_train_RMSE = np.sqrt(metrics.mean_squared_error(y_train, svr_init.predict(X_train)))
print('Initial modeling, train RMSE:', svr_init_train_RMSE)
svr_init_train_MAE = metrics.mean_absolute_error(y_train, svr_init.predict(X_train))
print('Initial modeling, train MAE:', svr_init_train_MAE)

Initial modeling, train RMSE: 278.6726219728049
Initial modeling, train MAE: 100.61073462141532


In [10]:
#Find the validation and validation RMSE and MAE
t2 = time.time()
svr_init_valid_RMSE = np.sqrt(metrics.mean_squared_error(y_valid, svr_init.predict(X_valid)))
t3 = time.time()
svr_init_predict_time = t3 - t2
print('Prediction time : ', svr_init_predict_time)
print('Initial modeling, validation RMSE:', svr_init_valid_RMSE)
svr_init_valid_MAE = metrics.mean_absolute_error(y_valid, svr_init.predict(X_valid))
print('Initial modeling, validation MAE:', svr_init_valid_MAE)

Prediction time :  7037.7495658397675
Initial modeling, validation RMSE: 274.4847653875946
Initial modeling, validation MAE: 99.31935905709184


In [11]:
results = {'model':['SVR preprocessed train', 'SVR preprocessed train', 'SVR preprocessed validation', 'SVR preprocessed validation'], 'error_type':['RMSE', 'MAE', 'RMSE', 'MAE'], 
           'score':[svr_init_train_RMSE, svr_init_train_MAE, svr_init_valid_RMSE, svr_init_valid_MAE]}

In [12]:
results_df = pd.DataFrame(results)

In [13]:
results_df.to_csv('../results/SVR_milestone_2')

## Catboost

### Raw data

### RMSE

In [5]:
results_df = pd.read_csv('../results/SVR_milestone_2')

In [6]:
# Initialize CatBoostRegressor
cat_init_raw_RMSE = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False, loss_function='RMSE')
# Fit model
t_cat0 = time.time()
cat_init_raw_RMSE.fit(X_train, y_train.to_numpy().ravel())
t_cat1 = time.time()

In [7]:
cat_init_train_time = t_cat1 - t_cat0
print('Training time : ', cat_init_train_time)

Training time :  57.30679130554199


In [8]:
#Find the train and validation RMSE
cat_init_train_RMSE = np.sqrt(metrics.mean_squared_error(y_train, cat_init_raw_RMSE.predict(X_train)))
print('Catboost Initial modeling, train RMSE:', cat_init_train_RMSE)
cat_init_valid_RMSE = np.sqrt(metrics.mean_squared_error(y_valid, cat_init_raw_RMSE.predict(X_valid)))
print('Catboost Initial modeling, validation RMSE:', cat_init_valid_RMSE)

Catboost Initial modeling, train RMSE: 39.67468516340541
Catboost Initial modeling, validation RMSE: 120.60530371795936


#### MAE

In [9]:
# Initialize CatBoostRegressor
cat_init_raw_MAE = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False, loss_function='MAE')
# Fit model
t_cat2 = time.time()
cat_init_raw_MAE.fit(X_train, y_train.to_numpy().ravel())
t_cat3 = time.time()

In [10]:
cat_init_train_time_MAE = t_cat3 - t_cat2
print('Training time : ', cat_init_train_time_MAE)

Training time :  62.7025146484375


In [11]:
#Find the train and validation MAE
cat_init_train_MAE = metrics.mean_absolute_error(y_train, cat_init_raw_MAE.predict(X_train))
print('Catboost Initial modeling, train MAE:', cat_init_train_MAE)
cat_init_valid_MAE = metrics.mean_absolute_error(y_valid, cat_init_raw_MAE.predict(X_valid))
print('Catboost Initial modeling, validation MAE:', cat_init_valid_MAE)

Catboost Initial modeling, train MAE: 28.706637185536987
Catboost Initial modeling, validation MAE: 53.97962785924548


In [12]:
results_df = results_df.append({'model':'catboost preprocessed train', 'error_type':'RMSE', 'score':cat_init_train_RMSE}, 
                               ignore_index=True)
results_df = results_df.append({'model':'catboost preprocessed train', 'error_type':'MAE', 'score':cat_init_train_MAE}, 
                               ignore_index=True)
results_df = results_df.append({'model':'catboost preprocessed validation', 'error_type':'RMSE', 'score':cat_init_valid_RMSE}, 
                               ignore_index=True)
results_df = results_df.append({'model':'catboost preprocessed validation', 'error_type':'MAE', 'score':cat_init_valid_RMSE}, 
                               ignore_index=True)
results_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,model,error_type,score
0,0.0,0.0,0.0,SVR preprocessed train,RMSE,278.672622
1,1.0,1.0,1.0,SVR preprocessed train,MAE,100.610735
2,2.0,2.0,2.0,SVR preprocessed validation,RMSE,274.484765
3,3.0,3.0,3.0,SVR preprocessed validation,MAE,99.319359
4,4.0,4.0,NaN,catboost preprocessed train,RMSE,37.583964
5,5.0,5.0,NaN,catboost preprocessed train,MAE,28.965351
6,6.0,6.0,NaN,catboost preprocessed validation,RMSE,105.144149
7,8.0,8.0,NaN,catboost preprocessed tuned train,RMSE,39.161389
8,9.0,9.0,NaN,catboost preprocessed tuned train,MAE,30.609809
9,10.0,10.0,NaN,catboost preprocessed tuned validation,RMSE,101.228868


In [22]:
results_df.to_csv('../results/SVR_milestone_2')

## Hyperparameters tuning : Random search

### RMSE

In [25]:
t_cat4 = time.time()
cat_tunned_RMSE = CatBoostRegressor(verbose=False, loss_function='RMSE')
parameters = {'iterations':[i for i in range(1,300,10)], 
              'learning_rate':np.logspace(-4,0,40), 
              'depth':[i for i in range(1,12,1)], 
              'bagging_temperature':[i for i in range(2,11)]}
catboost_opt_RMSE = RandomizedSearchCV(cat_tunned_RMSE, parameters, cv=5, iid=False, n_iter=30, n_jobs=-1, verbose=10)
catboost_opt_RMSE.fit(X_train, y_train.to_numpy().ravel())
t_cat5 = time.time()

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   57.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 50.6min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 70.2min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed: 83.6min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 94.0min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 110.3min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 115.5min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 124.1min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 137.1min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 141.9min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 144.9min finished
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/sk

In [27]:
print('running time optimisation : ', t_cat5 - t_cat4)
print('The best parameter combination is {}.'.format(catboost_opt_RMSE.best_params_))

running time optimisation :  8750.836293935776
The best parameter combination is {'learning_rate': 0.38881551803080855, 'iterations': 241, 'depth': 9, 'bagging_temperature': 4}.


In [28]:
#Find the train and validation RMSE
cat_tuned_train_RMSE = np.sqrt(metrics.mean_squared_error(y_train, catboost_opt_RMSE.predict(X_train)))
print('Catboost tuned, train RMSE:', cat_tuned_train_RMSE)
cat_tuned_valid_RMSE = np.sqrt(metrics.mean_squared_error(y_valid, catboost_opt_RMSE.predict(X_valid)))
print('Catboost tuned, validation RMSE:', cat_tuned_valid_RMSE)

Catboost tuned, train RMSE: 39.16138879609356
Catboost tuned, validation RMSE: 101.22886847723292


### MAE

In [29]:
t_cat6 = time.time()
cat_tunned_MAE = CatBoostRegressor(verbose=False, loss_function='MAE')
parameters = {'iterations':[i for i in range(1,300,10)], 
              'learning_rate':np.logspace(-4,0,40), 
              'depth':[i for i in range(1,12,1)], 
              'bagging_temperature':[i for i in range(2,11)]}
catboost_opt_MAE = RandomizedSearchCV(cat_tunned_MAE, parameters, cv=5, iid=False, n_iter=30, n_jobs=-1, verbose=10)
catboost_opt_MAE.fit(X_train, y_train.to_numpy().ravel())
t_cat7 = time.time()

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 12.2min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed: 17.6min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 18.5min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 19.0min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 23.8min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 25.0min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 32.2min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 43.6min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 79.0min finished
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/sklearn/

In [30]:
print('running time optimisation : ', t_cat6 - t_cat7)
print('The best parameter combination is {}.'.format(catboost_opt_MAE.best_params_))

running time optimisation :  -4932.689977407455
The best parameter combination is {'learning_rate': 0.30702906297578497, 'iterations': 81, 'depth': 11, 'bagging_temperature': 8}.


In [31]:
#Find the train and validation MAE
cat_tuned_train_MAE = metrics.mean_absolute_error(y_train, catboost_opt_MAE.predict(X_train))
print('Catboost tuned, train MAE:', cat_tuned_train_MAE)
cat_tuned_valid_MAE = metrics.mean_absolute_error(y_valid, catboost_opt_MAE.predict(X_valid))
print('Catboost tuned, validation MAE:', cat_tuned_valid_MAE)

Catboost tuned, train MAE: 30.60980854921523
Catboost tuned, validation MAE: 45.33389033213189


In [32]:
results_df = results_df.append({'model':'catboost preprocessed tuned train', 'error_type':'RMSE', 'score':cat_tuned_train_RMSE}, 
                               ignore_index=True)
results_df = results_df.append({'model':'catboost preprocessed tuned train', 'error_type':'MAE', 'score':cat_tuned_train_MAE}, 
                               ignore_index=True)
results_df = results_df.append({'model':'catboost preprocessed tuned validation', 'error_type':'RMSE', 'score':cat_tuned_valid_RMSE}, 
                               ignore_index=True)
results_df = results_df.append({'model':'catboost preprocessed tuned validation', 'error_type':'MAE', 'score':cat_tuned_valid_MAE}, 
                               ignore_index=True)
results_df

,Unnamed: 0,model,error_type,score
0,0.0,SVR preprocessed train,RMSE,278.672622
1,1.0,SVR preprocessed train,MAE,100.610735
2,2.0,SVR preprocessed validation,RMSE,274.484765
3,3.0,SVR preprocessed validation,MAE,99.319359
4,NaN,catboost preprocessed train,RMSE,37.583964
5,NaN,catboost preprocessed train,MAE,28.965351
6,NaN,catboost preprocessed validation,RMSE,105.144149
7,NaN,catboost preprocessed validation,RMSE,105.144149
8,NaN,catboost preprocessed tuned train,RMSE,39.161389
9,NaN,catboost preprocessed tuned train,MAE,30.609809


In [33]:
results_df.to_csv('../results/SVR_milestone_2')

### PCA

#### On the whole data set

In [4]:
X_train_pca = pca_fit_transform(X_train)
X_valid_pca = pca_transform(X_valid)

(9898, 632)
(632, 632)


In [6]:
print(X_train_pca.shape)
print(X_valid_pca.shape)

(39592, 356)
(9898, 356)


#### RMSE

In [7]:
# Initialize CatBoostRegressor
cat_init_raw_RMSE_pca = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False, loss_function='RMSE')
# Fit model
t_cat0_pca = time.time()
cat_init_raw_RMSE_pca.fit(X_train_pca, y_train.to_numpy().ravel())
t_cat1_pca = time.time()

In [8]:
cat_init_train_time_pca = t_cat1_pca - t_cat0_pca
print('Training time : ', cat_init_train_time_pca)

Training time :  19.16993498802185


In [9]:
#Find the train and validation RMSE
cat_init_train_RMSE_pca = np.sqrt(metrics.mean_squared_error(y_train, cat_init_raw_RMSE_pca.predict(X_train_pca)))
print('Catboost Initial modeling, train RMSE:', cat_init_train_RMSE_pca)
cat_init_valid_RMSE_pca = np.sqrt(metrics.mean_squared_error(y_valid, cat_init_raw_RMSE_pca.predict(X_valid_pca)))
print('Catboost Initial modeling, validation RMSE:', cat_init_valid_RMSE_pca)

Catboost Initial modeling, train RMSE: 42.16822078758682
Catboost Initial modeling, validation RMSE: 385.0706414400777


#### MAE

In [10]:
# Initialize CatBoostRegressor
cat_init_raw_MAE_pca = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False, loss_function='MAE')
# Fit model
t_cat2_pca = time.time()
cat_init_raw_MAE_pca.fit(X_train_pca, y_train.to_numpy().ravel())
t_cat3_pca = time.time()

In [11]:
cat_init_train_time_MAE_pca = t_cat3_pca - t_cat2_pca
print('Training time : ', cat_init_train_time_MAE_pca)

Training time :  19.42227816581726


In [12]:
#Find the train and validation MAE
cat_init_train_MAE_pca = metrics.mean_absolute_error(y_train, cat_init_raw_MAE_pca.predict(X_train_pca))
print('Catboost PCA modeling, train MAE:', cat_init_train_MAE_pca)
cat_init_valid_MAE_pca = metrics.mean_absolute_error(y_valid, cat_init_raw_MAE_pca.predict(X_valid_pca))
print('Catboost PCA modeling, validation MAE:', cat_init_valid_MAE_pca)

Catboost PCA modeling, train MAE: 39.045197421343026
Catboost PCA modeling, validation MAE: 310.25081079363355


In [13]:
results_df = pd.read_csv('../results/SVR_milestone_2')

In [14]:
results_df = results_df.append({'model':'catboost pca train', 'error_type':'RMSE', 'score':cat_init_train_RMSE_pca}, 
                               ignore_index=True)
results_df = results_df.append({'model':'catboost pca train', 'error_type':'MAE', 'score':cat_init_train_MAE_pca}, 
                               ignore_index=True)
results_df = results_df.append({'model':'catboost pca validation', 'error_type':'RMSE', 'score':cat_init_valid_RMSE_pca}, 
                               ignore_index=True)
results_df = results_df.append({'model':'catboost pca validation', 'error_type':'MAE', 'score':cat_init_valid_MAE_pca}, 
                               ignore_index=True)
results_df

,Unnamed: 0,Unnamed: 0.1,model,error_type,score
0,0.0,0.0,SVR preprocessed train,RMSE,278.672622
1,1.0,1.0,SVR preprocessed train,MAE,100.610735
2,2.0,2.0,SVR preprocessed validation,RMSE,274.484765
3,3.0,3.0,SVR preprocessed validation,MAE,99.319359
4,4.0,NaN,catboost preprocessed train,RMSE,37.583964
5,5.0,NaN,catboost preprocessed train,MAE,28.965351
6,6.0,NaN,catboost preprocessed validation,RMSE,105.144149
7,7.0,NaN,catboost preprocessed validation,RMSE,105.144149
8,8.0,NaN,catboost preprocessed tuned train,RMSE,39.161389
9,9.0,NaN,catboost preprocessed tuned train,MAE,30.609809


#### By groups

In [15]:
# drop rows missing target variable
df = drop_missing_unacast(df)
# create X and y
X = df.drop('unacast_session_count', axis=1)
y = df.loc[:, 'unacast_session_count']
# split the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                    test_size=0.2,
                                                      random_state=2020)
# impute NaN values
result = impute_data(X_train, X_valid)
X_train = result[0]
X_valid = result[1] 


# perform dropping
X_train = drop_columns(X_train)
X_valid = drop_columns(X_valid)

In [16]:
X_train_pca_gr = pca_fit_transform(X_train, by_groups=True)
X_valid_pca_gr = pca_transform(X_valid, by_groups=True)

In [21]:
X_train_pca_gr.shape

(39592, 406)

In [28]:
X_train_pca_gr = X_train_pca_gr.drop(columns=['income_class', 'density_class', 'climate']).set_axis([str(i) for i in range(403)], 
                                         axis=1, inplace=False)
X_valid_pca_gr = X_valid_pca_gr.drop(columns=['income_class', 'density_class', 'climate']).set_axis([str(i) for i in range(403)], 
                                         axis=1, inplace=False)

#### RMSE

In [29]:
# Initialize CatBoostRegressor
cat_init_raw_RMSE_pca_gr = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False, loss_function='RMSE')
# Fit model
t_cat0_pca_gr = time.time()
cat_init_raw_RMSE_pca_gr.fit(X_train_pca_gr, y_train.to_numpy().ravel())
t_cat1_pca_gr = time.time()

In [30]:
cat_init_train_time_pca_gr = t_cat1_pca_gr - t_cat0_pca_gr
print('Training time : ', cat_init_train_time_pca_gr)

Training time :  21.341124296188354


In [31]:
#Find the train and validation RMSE
cat_init_train_RMSE_pca_gr = np.sqrt(metrics.mean_squared_error(y_train, cat_init_raw_RMSE_pca_gr.predict(X_train_pca_gr)))
print('Catboost grouped PCA modeling, train RMSE:', cat_init_train_RMSE_pca_gr)
cat_init_valid_RMSE_pca_gr = np.sqrt(metrics.mean_squared_error(y_valid, cat_init_raw_RMSE_pca_gr.predict(X_valid_pca_gr)))
print('Catboost grouped PCA modeling, validation RMSE:', cat_init_valid_RMSE_pca_gr)

Catboost grouped PCA modeling, train RMSE: 38.15788227066586
Catboost grouped PCA modeling, validation RMSE: 624.7827432329105


#### MAE

In [32]:
# Initialize CatBoostRegressor
cat_init_raw_MAE_pca_gr = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False, loss_function='MAE')
# Fit model
t_cat2_pca_gr = time.time()
cat_init_raw_MAE_pca_gr.fit(X_train_pca_gr, y_train.to_numpy().ravel())
t_cat3_pca_gr = time.time()

In [33]:
cat_init_train_time_MAE_pca_gr = t_cat3_pca_gr - t_cat2_pca_gr
print('Training time : ', cat_init_train_time_MAE_pca_gr)

Training time :  21.70159339904785


In [34]:
#Find the train and validation MAE
cat_init_train_MAE_pca_gr = metrics.mean_absolute_error(y_train, cat_init_raw_MAE_pca_gr.predict(X_train_pca_gr))
print('Catboost grouped PCA modeling, train MAE:', cat_init_train_MAE_pca_gr)
cat_init_valid_MAE_pca_gr = metrics.mean_absolute_error(y_valid, cat_init_raw_MAE_pca_gr.predict(X_valid_pca_gr))
print('Catboost grouped PCA modeling, validation MAE:', cat_init_valid_MAE_pca_gr)

Catboost grouped PCA modeling, train MAE: 34.75919496561219
Catboost grouped PCA modeling, validation MAE: 395.2167765359392


In [35]:
results_df = results_df.append({'model':'catboost grouped PCA train', 'error_type':'RMSE', 'score':cat_init_train_RMSE_pca_gr}, 
                               ignore_index=True)
results_df = results_df.append({'model':'catboost grouped PCA train', 'error_type':'MAE', 'score':cat_init_train_MAE_pca_gr}, 
                               ignore_index=True)
results_df = results_df.append({'model':'catboost grouped PCA validation', 'error_type':'RMSE', 'score':cat_init_valid_RMSE_pca_gr}, 
                               ignore_index=True)
results_df = results_df.append({'model':'catboost grouped PCA validation', 'error_type':'MAE', 'score':cat_init_valid_MAE_pca_gr}, 
                               ignore_index=True)
results_df

,Unnamed: 0,Unnamed: 0.1,model,error_type,score
0,0.0,0.0,SVR preprocessed train,RMSE,278.672622
1,1.0,1.0,SVR preprocessed train,MAE,100.610735
2,2.0,2.0,SVR preprocessed validation,RMSE,274.484765
3,3.0,3.0,SVR preprocessed validation,MAE,99.319359
4,4.0,NaN,catboost preprocessed train,RMSE,37.583964
5,5.0,NaN,catboost preprocessed train,MAE,28.965351
6,6.0,NaN,catboost preprocessed validation,RMSE,105.144149
7,7.0,NaN,catboost preprocessed validation,RMSE,105.144149
8,8.0,NaN,catboost preprocessed tuned train,RMSE,39.161389
9,9.0,NaN,catboost preprocessed tuned train,MAE,30.609809


In [36]:
results_df.to_csv('../results/SVR_milestone_2')

## Drop same columns as in MLE

In [6]:
to_delete = ['monthly_repeated_sessions', 'B23008e21', 'B23008e23', 'B12001e11', 'B23008e26', 'B11005e14', 'B11005e17', 
             'B20004e9', 'B19101e7', 'B19101e4', 'B12001e15', 'B09002e19', 'B23008e27', 'B20004e7', 'B11016e10', 'B17012e31', 
             'B19101e8', 'C18108e5', 'B25012e8', 'B25012e15', 'B17020e4', 'B25012e16', 'B25012e10', 'B25012e6', 'B09002e17',
             'B25012e12', 'B25012e11', 'B08301e6', 'B17012e6', 'B19101e6', 'B25012e4', 'B19101e12', 'B17020e5', 'B23008e18', 
             'B19101e5', 'B12001e16', 'B08303e12', 'B17020e11', 'B10010e1', 'B19101e11', 'B08303e13', 'B16007e5', 'B19101e16', 
             'B11016e12', 'B17020e3', 'B19101e3', 'B25012e14', 'B09002e18', 'B17012e2', 'B19101e2', 'B25012e5', 'B17012e5', 
             'B10010e2', 'B11005e4', 'B19101e15', 'B11005e5', 'B19101e17', 'B17012e4', 'B17012e7', 'B20004e8', 'B17020e10', 
             'B09018e8', 'B20004e6', 'B17020e6', 'B10002e5', 'B11016e11', 'B10002e3', 'B17012e26', 'B25012e3', 'B11005e10', 
             'B08301e5', 'B17012e25', 'B17012e20', 'B08301e4', 'B19101e13', 'B10002e4', 'B23008e13', 'C18108e3', 'B09018e7', 
             'B11001e9', 'B10002e1', 'C18108e4', 'B23008e19', 'B10010e3', 'B16007e7', 'B23008e12', 'B15003e23', 'B15003e25', 
             'B11005e6', 'B11003e7', 'B27003e1', 'B15003e20', 'B09018e1', 'B11016e3', 'B01001e4', 'B11016e2', 'B15003e21', 
             'B15003e22', 'B19083e1', 'B15003e24', 'B23020e1', 'B11016e4', 'B09018e5', 'B19125e2', 'B16007e3', 'B01001e28', 
             'B23020e3', 'B23020e2', 'B12001e6', 'B27001e30', 'B01001e29', 'B25103e2', 'B16007e4', 'B12001e9', 'B23008e10', 
             'B09018e4', 'B01001e30', 'B11003e5', 'B23008e2', 'B23008e5', 'B09002e16', 'B25012e9', 'B01002e3', 'B20004e5', 
             'B25012e2', 'B08303e1', 'B23025e4', 'B23008e4', 'B23025e7', 'B12001e1', 'B15003e17', 'B11005e7', 'B01001e3', 
             'B23025e2', 'B09002e8', 'B22003e5', 'B08301e1', 'B14002e1', 'B09002e11', 'B27001e2', 'B25064e1', 'B25001e1', 
             'B19058e2', 'B09001e5', 'B09002e10', 'B09002e13', 'B09002e12', 'B19301e1', 'B25119e2', 'B19125e3', 'B13016e4', 
             'B11001e5', 'B25119e3', 'B15003e18', 'B01001e27', 'B13016e5', 'B11005e9', 'B20004e2', 'B15003e1', 'B10001e4', 
             'B11001e6', 'B13016e7', 'B12001e7', 'B23008e9', 'B13016e6', 'B23008e7', 'B23008e6', 'B09002e14', 'B09001e10', 
             'B11005e8', 'B13016e3', 'male_60_69', 'male_80_over', 'female_22_29', 'female_30_39', 'female_40_49', 
             'female_50_59', 'female_60_69', 'female_70_79', 'female_80_over', 'total_enrolled', 'not_enrolled', 
             'enrolled_nursery_pre_private', 'enrolled_kinder_public', 'enrolled_kinder_private', 'enrolled_grades_1_4_public', 
             'enrolled_grades_1_4_private', 'enrolled_grades_5_8_public', 'enrolled_grades_9_12_private', 
             'enrolled_undergrad_public', 'enrolled_undergrad_private', 'enrolled_graduate_public', 'less_than_12_no_diploma', 
             'travel_15_minutes_less', 'travel_15_29_minutes', 'travel_30_44_minutes', 'single_under_6', 'single_0_to_17', 
             'single_6_to_17', 'single_no_kids', 'four_or_more_in_family_household', 'four_or_more_in_nonfamily_household', 
             'men_without_health_insurance', 'women_with_health_insurnace', 'women_without_health_insurance', 
             'avg_birth_weight', 'avg_age_of_mother', 'longitude', 'latitude', 'alcohol', 'amenity', 'bank', 'bar', 'cafe', 
             'camp_site', 'car_repair', 'childcare', 'clothes_store', 'convenience_store', 'fast_food', 'fire_station', 
             'fitness_or_sports_centre', 'fuel', 'healthcare', 'hotel', 'museum_or_gallery', 'restaurant', 'shop', 
             'supermarket', 'tourism', 'distance_to_U', 'distance_to_nearest_school', 'walk_score', 'bike_score', 'k_avg', 
             'streets_per_node_avg', 'edge_length_total', 'edge_length_avg', 'street_length_avg', 
             'streets_per_node_counts_2', 'streets_per_node_counts_3', 'streets_per_node_counts_5', 'n_osdw', 
             'k_avg_osdw', 'streets_per_node_avg_osdw', 'edge_length_total_osdw', 'streets_per_node_counts_1_osdw', 
             'streets_per_node_counts_3_osdw', 'streets_per_node_counts_5_osdw', 'intersection_count_osid', 
             'edge_length_total_osid', 'edge_length_avg_osid', 'street_length_total_osid', 'street_segments_count_osid', 
             'streets_per_node_counts_1_osid', 'streets_per_node_counts_2_osid', 'streets_per_node_counts_4_osid', 
             'streets_per_node_counts_5_osid', 'violent_crime', 'criminal_homicide', 'rape', 'robbery', 
             'aggravated_assault', 'property_crime', 'burglary', 'larceny_theft', 'motor_vehicle_theft', 
             'houses_per_sq_km', 'historic_number_of_sessions', 'historic_sessions_per_day', 'historic_unique_sessions', 
             'historic_unique_sessions_per_day', 'historic_repeat_sessions', 'historic_repeat_sessions_per_day', 
             'historic_total_session_length', 'historic_avg_session_length', 'historic_avg_light_activity', 
             'historic_avg_moderate_activity', 'historic_avg_vigorous_activity', 'historic_avg_mod_plus_vig', 
             'historic_hour_7', 'historic_hour_8', 'historic_hour_9', 'historic_hour_10', 'historic_hour_11', 
             'historic_hour_13', 'historic_hour_14', 'historic_hour_15', 'historic_hour_16', 'historic_hour_17', 
             'historic_hour_19', 'historic_hour_20', 'historic_rain', 'historic_foggy', 'historic_snow', 'Green_2016', 
             'Libertarians_2016', 'Poor_physical_health_days', 'Adult_smoking', 'Adult_obesity', 'weather_clear', 
             'weather_rain', 'weather_fog', 'temp_avg_35_below', 'temp_max_35_below', 'temp_max_45_55', 
             'state_amount_per_capita', 'historic_slide_count_comb', 'monthly_climb_count_comb', 'monthly_tube_count_comb', 
             'historic_tube_count_comb', 'monthly_overhang_count_comb', 'historic_overhang_count_comb', 
             'monthly_bridge_count_comb', 'historic_bridge_count_comb', 'monthly_swing_count_comb', 
             'historic_swing_count_comb', 'historic_obsta_count_comb', 'historic_crawls_count_comb', 
             'monthly_hour_night', 'historic_hour_night', 'avg_wind_calm', 'avg_wind_light_air', 'avg_wind_light_br', 
             'avg_wind_gentle_br', 'avg_wind_moderate_br', 'monthly_ws_calm', 'monthly_ws_light_air', 'monthly_ws_light_br', 
             'monthly_ws_gentle_br', 'monthly_ws_moderate_br', 'historic_ws_calm', 'historic_ws_light_air', 
             'historic_ws_light_br', 'historic_ws_gentle_br', 'historic_ws_moderate_br', 'avg_fertility_rate', 'HI', 'LI', 
             'MI', 'HD', 'LD', 'MD', 'monthly_repeated_sessions', 'historic_repeat_sessions']
X_train = X_train.drop(columns = to_delete)
X_valid = X_valid.drop(columns=to_delete)

In [15]:
# Initialize CatBoostRegressor
cat_init_raw_RMSE = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False, loss_function='RMSE')
# Fit model
t_cat0 = time.time()
cat_init_raw_RMSE.fit(X_train, y_train.to_numpy().ravel())
t_cat1 = time.time()

In [16]:
#Find the train and validation RMSE
cat_init_train_RMSE = np.sqrt(metrics.mean_squared_error(y_train, cat_init_raw_RMSE.predict(X_train)))
print('Catboost Initial modeling, train RMSE:', cat_init_train_RMSE)
cat_init_valid_RMSE = np.sqrt(metrics.mean_squared_error(y_valid, cat_init_raw_RMSE.predict(X_valid)))
print('Catboost Initial modeling, validation RMSE:', cat_init_valid_RMSE)

Catboost Initial modeling, train RMSE: 37.86621550275267
Catboost Initial modeling, validation RMSE: 106.87973865467465


In [ ]:
#Fit basic SVR with default values
t0 = time.time()
svr_init = SVR()
svr_init.fit(X_train, y_train.to_numpy().ravel())
t1 = time.time()

In [ ]:
svr_init_train_time = t1 - t0
print('Training time : ', svr_init_train_time)

In [ ]:
#Find the train RMSE and MAE
svr_init_train_RMSE = np.sqrt(metrics.mean_squared_error(y_train, svr_init.predict(X_train)))
print('Initial modeling, train RMSE:', svr_init_train_RMSE)
svr_init_train_MAE = metrics.mean_absolute_error(y_train, svr_init.predict(X_train))
print('Initial modeling, train MAE:', svr_init_train_MAE)

In [17]:
# Initialize CatBoostRegressor
cat_init_raw_MAE = CatBoostRegressor(iterations=100, learning_rate=1, depth=10, verbose=False, loss_function='MAE')
# Fit model
t_cat2 = time.time()
cat_init_raw_MAE.fit(X_train, y_train.to_numpy().ravel())
t_cat3 = time.time()

In [18]:
#Find the train and validation MAE
cat_init_train_MAE = metrics.mean_absolute_error(y_train, cat_init_raw_MAE.predict(X_train))
print('Catboost Initial modeling, train MAE:', cat_init_train_MAE)
cat_init_valid_MAE = metrics.mean_absolute_error(y_valid, cat_init_raw_MAE.predict(X_valid))
print('Catboost Initial modeling, validation MAE:', cat_init_valid_MAE)

Catboost Initial modeling, train MAE: 30.552682106051552
Catboost Initial modeling, validation MAE: 54.62643734321912


In [19]:
t_cat4 = time.time()
cat_tunned_RMSE = CatBoostRegressor(verbose=False, loss_function='RMSE')
parameters = {'iterations':[i for i in range(1,300,10)], 
              'learning_rate':np.logspace(-4,0,40), 
              'depth':[i for i in range(1,12,1)], 
              'bagging_temperature':[i for i in range(2,11)]}
catboost_opt_RMSE = RandomizedSearchCV(cat_tunned_RMSE, parameters, cv=5, iid=False, n_iter=10, n_jobs=-1, verbose=10)
catboost_opt_RMSE.fit(X_train, y_train.to_numpy().ravel())
t_cat5 = time.time()

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:  3.0min remaining:   40.0s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:  6.5min remaining:   24.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  6.5min finished
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:849: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


In [20]:
#Find the train and validation RMSE
cat_tuned_train_RMSE = np.sqrt(metrics.mean_squared_error(y_train, catboost_opt_RMSE.predict(X_train)))
print('Catboost tuned, train RMSE:', cat_tuned_train_RMSE)
cat_tuned_valid_RMSE = np.sqrt(metrics.mean_squared_error(y_valid, catboost_opt_RMSE.predict(X_valid)))
print('Catboost tuned, validation RMSE:', cat_tuned_valid_RMSE)

Catboost tuned, train RMSE: 90.52821487664427
Catboost tuned, validation RMSE: 120.15684771599464


In [21]:
print('running time optimisation : ', t_cat5 - t_cat4)
print('The best parameter combination is {}.'.format(catboost_opt_RMSE.best_params_))

running time optimisation :  414.15146350860596
The best parameter combination is {'learning_rate': 0.11937766417144358, 'iterations': 91, 'depth': 11, 'bagging_temperature': 2}.


In [22]:
t_cat4 = time.time()
cat_tunned_RMSE = CatBoostRegressor(verbose=False, loss_function='RMSE')
parameters = {'iterations':[i for i in range(1,300,10)],
              'depth':[i for i in range(1,12,1)], 
              'bagging_temperature':[i for i in range(2,11)]}
catboost_opt_RMSE = RandomizedSearchCV(cat_tunned_RMSE, parameters, cv=5, iid=False, n_iter=10, n_jobs=-1, verbose=10)
catboost_opt_RMSE.fit(X_train, y_train.to_numpy().ravel())
t_cat5 = time.time()

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   59.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:  4.6min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:  5.2min remaining:   20.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  5.4min finished
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:849: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


In [23]:
#Find the train and validation RMSE
cat_tuned_train_RMSE = np.sqrt(metrics.mean_squared_error(y_train, catboost_opt_RMSE.predict(X_train)))
print('Catboost tuned, train RMSE:', cat_tuned_train_RMSE)
cat_tuned_valid_RMSE = np.sqrt(metrics.mean_squared_error(y_valid, catboost_opt_RMSE.predict(X_valid)))
print('Catboost tuned, validation RMSE:', cat_tuned_valid_RMSE)

Catboost tuned, train RMSE: 52.01028652982752
Catboost tuned, validation RMSE: 100.4223670622564


In [24]:
print('running time optimisation : ', t_cat5 - t_cat4)
print('The best parameter combination is {}.'.format(catboost_opt_RMSE.best_params_))

running time optimisation :  345.98622727394104
The best parameter combination is {'iterations': 291, 'depth': 9, 'bagging_temperature': 5}.


In [36]:
t_cat4 = time.time()
cat_tunned_RMSE = CatBoostRegressor(verbose=False, loss_function='RMSE')
parameters = {'iterations':[i for i in range(1,300,10)],
              'depth':[i for i in range(1,15,1)]}
catboost_opt_RMSE = RandomizedSearchCV(cat_tunned_RMSE, parameters, cv=5, iid=False, n_iter=10, n_jobs=-1, verbose=10)
catboost_opt_RMSE.fit(X_train, y_train.to_numpy().ravel())
t_cat5 = time.time()

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done  24 out of  35 | elapsed:  5.8min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done  28 out of  35 | elapsed:  6.0min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  32 out of  35 | elapsed: 13.3min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed: 13.3min finished
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/sklearn/model_selection/_search.py:849: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


In [37]:
#Find the train and validation RMSE
cat_tuned_train_RMSE = np.sqrt(metrics.mean_squared_error(y_train, catboost_opt_RMSE.predict(X_train)))
print('Catboost tuned, train RMSE:', cat_tuned_train_RMSE)
cat_tuned_valid_RMSE = np.sqrt(metrics.mean_squared_error(y_valid, catboost_opt_RMSE.predict(X_valid)))
print('Catboost tuned, validation RMSE:', cat_tuned_valid_RMSE)

Catboost tuned, train RMSE: 32.11514830001279
Catboost tuned, validation RMSE: 98.02511090062508


In [38]:
print('running time optimisation : ', t_cat5 - t_cat4)
print('The best parameter combination is {}.'.format(catboost_opt_RMSE.best_params_))

running time optimisation :  907.6330139636993
The best parameter combination is {'iterations': 201, 'depth': 12}.


In [10]:
t_cat4 = time.time()
cat_tunned_RMSE = CatBoostRegressor(verbose=False, loss_function='RMSE')
parameters = {'iterations':[i for i in range(1,300,10)], 
              'depth':[i for i in range(1,14,1)], 
              'bagging_temperature':[i for i in range(2,11)]}
catboost_opt_RMSE = RandomizedSearchCV(cat_tunned_RMSE, parameters, cv=5, iid=False, n_iter=30, n_jobs=-1, verbose=10)
catboost_opt_RMSE.fit(X_train, y_train.to_numpy().ravel())
t_cat5 = time.time()

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 17.1min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 19.0min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 65.3min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 92.5min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 99.9min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 112.6min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 181.8min finished
/home/ubuntu/.local/share/virtualenvs/capstone_machine_learning-9cUIoDBU/lib/python3.7/site-packages/sklear

In [11]:
#Find the train and validation RMSE
cat_tuned_train_RMSE = np.sqrt(metrics.mean_squared_error(y_train, catboost_opt_RMSE.predict(X_train)))
print('Catboost tuned, train RMSE:', cat_tuned_train_RMSE)
cat_tuned_valid_RMSE = np.sqrt(metrics.mean_squared_error(y_valid, catboost_opt_RMSE.predict(X_valid)))
print('Catboost tuned, validation RMSE:', cat_tuned_valid_RMSE)

Catboost tuned, train RMSE: 23.0217249994348
Catboost tuned, validation RMSE: 100.29306364596661


In [12]:
print('running time optimisation : ', t_cat5 - t_cat4)
print('The best parameter combination is {}.'.format(catboost_opt_RMSE.best_params_))

running time optimisation :  11622.70536351204
The best parameter combination is {'iterations': 201, 'depth': 13, 'bagging_temperature': 6}.
